In [9]:
import pandas as pd
import numpy as np
import pdfplumber
import re
import glob
import unicodedata
import os
from tqdm import tqdm

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [17]:
def slugify(text: str) -> str:
    text = unicodedata.normalize('NFKD', text)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    text = text.lower()
    text = re.sub(r'[^a-z0-9]+', '_', text)
    return re.sub(r'_+', '_', text).strip('_')

def normalize_question(q: str) -> str:
    """
    Normaliza preguntas que varían por contenidos entre paréntesis u otras variables,
    reemplazando patrones específicos por una versión genérica.
    """
    # Patrón para la pregunta de ciudadanía con country variable
    citizen_pat = re.compile(
        r"^Is this applicant a citizen of the selected country of passport.*\?$", 
        re.IGNORECASE
    )
    if citizen_pat.match(q):
        return "Is this applicant a citizen of the selected country of passport?"
    # Añade otros patrones de normalización aquí si es necesario
    return q

def extract_questions_answers(pdf_path: str) -> list:
    """
    Extrae lista de (pregunta_normalizada, respuesta) del PDF,
    uniendo fragmentos multi‑línea y saltándose secciones y campos inline.
    """
    section_re = re.compile(r'^[A-Z][A-Za-z ]+$')
    inline_re  = re.compile(r'^(.+?):')
    
    # 1) Leer todas las líneas
    with pdfplumber.open(pdf_path) as pdf:
        lines = []
        for p in pdf.pages:
            lines.extend((p.extract_text() or '').split('\n'))
    lines = [l.strip() for l in lines if l.strip()]
    
    qa = []
    i = 0
    while i < len(lines):
        # 2) Detectar fin de pregunta
        if lines[i].endswith('?'):
            # 3) Retroceder para juntar cualquier fragmento previo
            j = i - 1
            question_fragments = [lines[i]]
            while j >= 0:
                prev = lines[j]
                # si es inline o sección o termina en '?' dejamos de retroceder
                if inline_re.match(prev) or section_re.match(prev) or prev.endswith('?'):
                    break
                # si no, es parte de la pregunta -> insertarlo al inicio
                question_fragments.insert(0, prev)
                j -= 1
            
            full_q = " ".join(question_fragments)
            # 4) Buscar la respuesta (primera línea no‑pregunta tras el '?')
            k = i + 1
            while k < len(lines) and lines[k].endswith('?'):
                k += 1
            answer = lines[k] if k < len(lines) else ""
            
            # 5) Normalizar y guardar
            norm_q = normalize_question(full_q)
            qa.append((norm_q, answer))
            
            # 6) Saltar past la respuesta
            i = k + 1
            continue
        
        i += 1
    
    return qa

def extract_fields_with_context(pdf_path: str) -> dict:
    data = {}
    current_section = None
    section_pattern = re.compile(r'^[A-Z][A-Za-z ]+$')
    inline_pattern  = re.compile(r'^(.+?):\s*(.+)$')

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            for line in page.extract_text().split('\n'):
                line = line.strip()
                if not line:
                    continue
                if section_pattern.match(line) and ':' not in line and not line.endswith('?'):
                    current_section = slugify(line)
                    continue
                m = inline_pattern.match(line)
                if m:
                    field, val = m.groups()
                    key = slugify(field)
                    if current_section:
                        key = f"{current_section}_{key}"
                    data[key] = val.strip()
    return data

In [18]:
def process_application_pdfs(root_folder: str, question_map=None):
    """
    Procesa subcarpetas numeradas en root_folder.
    - question_map: dict existente {idx: pregunta}. Si es None, se inicia vacío.
    Devuelve (DataFrame, updated_question_map).
    """
    if question_map is None:
        question_map = {}
    rows = []

    items_list = [d for d in set(os.listdir(root_folder)) if d != '.DS_Store']

    for sample_dir in tqdm(sorted(items_list, key=lambda x: int(x) if x.isdigit() else x)):
        folder = os.path.join(root_folder, sample_dir)
        if not os.path.isdir(folder): continue
        # renombrar archivos a minúsculas
        for fname in os.listdir(folder):
            old, new = os.path.join(folder,fname), os.path.join(folder,fname.lower())
            if old != new: os.rename(old,new)
        # buscar PDF
        cands = [f for f in os.listdir(folder) if 'application' in f and f.endswith('.pdf')]
        if not cands: continue
        pdf_path = os.path.join(folder, cands[0])
        inline_data = extract_fields_with_context(pdf_path)
        qa_list = extract_questions_answers(pdf_path)
        # actualizar mapping
        for q, _ in qa_list:
            if q not in question_map.values():
                new_idx = max(question_map.keys(), default=0) + 1
                question_map[new_idx] = q
        # construir fila
        row = dict(inline_data)
        row['sample_id'] = sample_dir
        row['source_file'] = os.path.basename(pdf_path)
        # inicializar columnas de respuestas
        for idx in question_map:
            row[f'question_{idx}'] = None
        # asignar respuestas
        for idx, q_text in question_map.items():
            for q, a in qa_list:
                if q == q_text:
                    row[f'question_{idx}'] = a
                    break
        rows.append(row)
    df = pd.DataFrame(rows)
    return df, question_map


In [19]:
# Ruta raíz donde están las carpetas numeradas

df_all_granted, question_map_g = process_application_pdfs('data/granted/')
df_all_granted['final_visa_status'] = 'granted'

print(len(question_map_g))

df_all_refused, question_map_r = process_application_pdfs('data/refused/', question_map = question_map_g)
df_all_refused['final_visa_status'] = 'refused'

100%|██████████| 4/4 [00:30<00:00,  7.54s/it]


66


100%|██████████| 4/4 [00:33<00:00,  8.39s/it]


In [20]:
len(question_map_r)

81

In [26]:
question_map_r[69]

'Does this family member have a national identity card?'

In [30]:
df_all_refused.shape#[['question_69','question_70','question_80','question_81']]

(4, 271)

In [28]:
df_all = pd.concat([df_all_granted,df_all_refused])

In [29]:
df_all

,official,current_location_current_location,current_location_legal_status,confirmation_of_enrolment_details_coe_code,no_note,personal_privacy_official,personal_privacy_generated,education_sector_education_sector,primary_applicant_information,passport_details_family_name,passport_details_given_names,passport_details_sex,passport_details_date_of_birth,passport_details_passport_number,passport_details_country_of_passport,passport_details_nationality_of_passport_holder,passport_details_date_of_issue,passport_details_date_of_expiry,national_identity_card_family_name,application_for_a_student_visa_given_names,application_for_a_student_visa_identification_number,application_for_a_student_visa_country_of_issue,application_for_a_student_visa_note,application_for_a_student_visa_date_of_issue,application_for_a_student_visa_date_of_expiry,place_of_birth_town_city,place_of_birth_state_province,place_of_birth_country_of_birth,relationship_status_relationship_status,application_for_a_student_visa_family_name,application_for_a_student_visa_sex,application_for_a_student_visa_date_of_birth,application_for_a_student_visa_country_of_birth,application_for_a_student_visa_passport_number,application_for_a_student_visa_country_of_passport,country_of_residence_usual_country_of_residence,department_office_office,residential_address_country,residential_address_address,residential_address_suburb_town,application_for_a_student_visa_state_or_province,application_for_a_student_visa_postal_code,contact_telephone_numbers_home_phone,email_address_email_address,education_agent_family_name,education_agent_given_names,education_agent_organisation,postal_address_country,postal_address_address,postal_address_suburb_town,postal_address_state_or_province,postal_address_postal_code,contact_telephone_numbers_business_phone,the_department_prefers_to_communicate_electronically_as_this_provides_a_faster_method_of_email_address,the_department_prefers_to_communicate_electronically_as_this_provides_a_faster_method_of_note,family_member_details_family_name,family_member_details_given_names,family_member_details_sex,family_member_details_date_of_birth,usual_country_of_residence_usual_country_of_residence,no_funding_type,provide_details_of_the_the_money_for_my_experience_comes_from_my_own_savings_from_ongoing_income_example,no_name_of_health_insurer,no_insurance_policy_number,no_date_from,no_date_to,highest_level_of_schooling_course_name,highest_level_of_schooling_institution_name,highest_level_of_schooling_country_of_institution,employment_history_details_employment_status,application_for_a_student_visa_organisation_name,application_for_a_student_visa_industry_type,organisation_address_country,organisation_address_address,organisation_address_suburb_town,organisation_address_state_or_province,organisation_address_postal_code,contact_person_details_family_name,contact_person_details_given_names,contact_person_details_business_phone,position_details_position,position_details_date_from,no_organisation_name,no_industry_type,application_for_a_student_visa_suburb_town,position_details_date_to,student_course_details_course_name,student_course_details_institution_name,student_course_details_date_from,student_course_details_date_to,main_language_main_language,yes_give_details,no_has_any_applicant_ever_been_charged_with_or_indicted_for,student_declarations_have_read_and_understood_the_information_provided_on_the_website_https,sample_id,source_file,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,question_9,question_10,question_11,question_12,question_13,question_14,question_15,question_16,question_17,question_18,question_19,question_20,question_21,question_22,question_23,question_24,question_25,question_26,question_27,question_28,question_29,question_30,question_31,question_32,question_33,question_34,question_35,question_36,question_37,question_38,question_39,question_40,question_41,question_42,question_43,question_44,question_45,question_46,questi